In [1]:
import wandb
import rna_model
import utils
import os
from transformers import LongformerConfig, LongformerForMaskedLM
from transformers import Trainer, TrainingArguments
from dna_tokenizer import rnabert_maskwrapper,DNATokenizer
os.CUDA_VISIBLE_DEVICES='0'

In [2]:
tokenizer = DNATokenizer('./vocab/base_vocab.txt',max_len=3072)
data_dir = '../data/pre-train/3070/rna_seq.h5'
train_data = rna_model.rna_self_mask(data_dir,'train',tokenizer,max_length=3072)
valid_data = rna_model.rna_self_mask(data_dir,'valid',tokenizer,max_length=3072)
data_collator = rnabert_maskwrapper(tokenizer,0.15,extend = False)

In [3]:
configuration = LongformerConfig(vocab_size = tokenizer.vocab_size,
                                pad_token_id = tokenizer.pad_token_id,
                                eos_token_id = tokenizer.sep_token_id,
                                bos_token_id = tokenizer.cls_token_id,
                                type_vocab_size = 1,
                                layer_norm_eps = 1e-05,
                                max_position_embeddings = 3062
                                )

In [4]:
model = LongformerForMaskedLM(configuration)

In [5]:
model.eval()

LongformerForMaskedLM(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(9, 768, padding_idx=0)
      (position_embeddings): Embedding(3062, 768, padding_idx=0)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0): LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_global): Linear

In [9]:
training_args = TrainingArguments(
        output_dir = './longformer_test/',
        num_train_epochs=70,
        do_train=True,
        learning_rate = 6e-04,
        adam_beta1 = 0.9,
        adam_beta2 = 0.98,
        adam_epsilon = 1e-06,
        weight_decay = 0.01,
        warmup_steps = 500,
        lr_scheduler_type = 'linear',
        evaluation_strategy = 'steps',
        gradient_accumulation_steps = 40,
        per_device_train_batch_size=1,
        logging_steps = 80,
        eval_steps = 500,
        save_total_limit=2,
        ddp_find_unused_parameters = False,
        load_best_model_at_end=True
    )

In [10]:
trainer = Trainer(model = model, 
                args = training_args, 
                train_dataset=train_data, 
                eval_dataset=valid_data,
                data_collator=data_collator)

In [11]:
trainer.train()

/home/amber/tf_2/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 604629
  Num Epochs = 70
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 320
  Gradient Accumulation steps = 40
  Total optimization steps = 132230
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ambert (use `wandb login --relogin` to force relogin)


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [1]:
import h5py

In [ ]:
file = h5py.File('../data/species/combined_vds.h5','r')